In [10]:
import pandas as pd
import numpy as np

train_data = pd.read_pickle('./data/train_data.pkl')
test_data = pd.read_pickle('./data/test_data.pkl')
train_lable = pd.read_pickle('./data/train_lable.pkl')

train_data = train_data.drop(['PassengerId', 'Fare_stand', 'Fare_minmax'], axis=1)
test_id = test_data[['PassengerId']]
test_data = test_data.drop(['PassengerId', 'Fare_stand', 'Fare_minmax'], axis=1)

print(train_data.shape)
print(test_data.shape)

(891, 42)
(418, 42)


In [11]:
train_data.head()

,Age,SibSp,Parch,Fare,ticket_cnt,Family,Fare_log,Pclass_1,Pclass_2,Pclass_3,...,Embarked_S,Cabin_class_-,Cabin_class_A,Cabin_class_B,Cabin_class_C,Cabin_class_D,Cabin_class_E,Cabin_class_F,Cabin_class_G,Cabin_class_T
0,22.0,1,0,7.2500,1,1,2.110213,0,0,1,...,1,1,0,0,0,0,0,0,0,0
1,38.0,1,0,71.2833,2,1,4.280593,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,26.0,0,0,7.9250,1,0,2.188856,0,0,1,...,1,1,0,0,0,0,0,0,0,0
4,35.0,1,0,53.1000,2,1,3.990834,1,0,0,...,1,0,0,0,1,0,0,0,0,0
6,35.0,0,0,8.0500,1,0,2.202765,0,0,1,...,1,1,0,0,0,0,0,0,0,0


這次的建模主要採用ensemble的模型，ensemble的方式是採用多個錯誤率小於0.5的模型進行投票，如果模型之間獨立的話可以得到更低的錯誤率。

常見的方式有Bagging跟Boosting:

Bagging為統計上常見的boostrap aggregation方法，從資料集中進行重複取樣，有一個樣本取到的機率近似於63%。
隨機森林為常見的Bagging模型，會針對資料集和特徵進行隨機採樣，並且不進行樹的剪裁，優點為隨機取樣，因此比較不會受雜訊影響產生overfitting的問題，可以降低模型的不穩定性。


In [12]:
x_train, x_test, y_train, y_test = train_test_split(train_data, train_lable, test_size=0.25, random_state=4)

In [33]:
from xgboost import XGBClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split

xgbc = XGBClassifier()
xgbc.fit(x_train, y_train)
y_pred1 = xgbc.predict(x_test)

print(xgbc.score(x_test, y_test))
      
clf = RandomForestClassifier(random_state=5, n_estimators=200, min_samples_split=20, oob_score=True, n_jobs=2)
clf.fit(x_train, y_train)
y_pred2 = clf.predict(x_test)
acc = metrics.accuracy_score(y_test, y_pred2)
print("Accuracy: ", acc)
      
lr = LogisticRegression()
lr.fit(x_train, y_train)
y_pred3 = lr.predict(x_test)
acc = metrics.accuracy_score(y_test, y_pred3)
print("Accuracy: ", acc)

0.874439461883408
Accuracy:  0.8609865470852018
Accuracy:  0.8654708520179372


D:\Anacoda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [34]:
# 0.75598
xgbc = XGBClassifier()
xgbc.fit(train_data, train_lable)
y_pred1 = xgbc.predict_proba(test_data)[:,1]

# 0.79904
clf = RandomForestClassifier(random_state=5, n_estimators=200, min_samples_split=20, oob_score=True, n_jobs=2)
clf.fit(train_data, train_lable)
y_pred2 = clf.predict_proba(test_data)[:,1]

# 0.77511
lr = LogisticRegression()
lr.fit(train_data, train_lable)
y_pred3 = lr.predict_proba(test_data)[:,1]

D:\Anacoda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [29]:
# use blending
pred = y_pred1*0.32 + y_pred2*0.35 + y_pred3*0.33
pred = np.round(pred,0).astype(int)

test_id['Survived'] = list(pred)
test_id.to_csv('blendding.csv', index = False)

In [38]:
from mlxtend.classifier import StackingClassifier

meta_estimator = GradientBoostingClassifier(tol=100, subsample=0.70, n_estimators=50, 
                                           max_features='sqrt', max_depth=4, learning_rate=0.3)

stacking = StackingClassifier(classifiers=[xgbc, clf, lr], meta_classifier=meta_estimator)
stacking.fit(train_data, train_lable)
stacking_pred = stacking.predict(test_data)
test_id['Survived'] = list(stacking_pred)
test_id.to_csv('titanic_stacking.csv', index = False)

D:\Anacoda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
